# 거래

In [ ]:
import pandas as pd
import pymysql
from datetime import datetime
import traceback

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### Back test 파라미터
## 투자금 / 분할
totalAmount, step = 10000000, 5
tradeAmount = totalAmount / step
## Boundary
position_bid = 0.1
position_ask = 0.9
mfi_bid = 10
mfi_ask = 90

### 1. 종목(cap >= 5천억 && KOSPI)조회
print(">> 1. 종목(cap >= 5천억 && KOSPI)조회")

# 1.0. 매수종목조회
print("1.0. 매수종목조회")
cursor.execute("SELECT * FROM bid_basket")
basketDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'volume', 3:'price', 4:'mfi', 5:'position'})
basketDf.set_index('ticker', drop=True, inplace=True)

# 1.1. 기간 설정
print("1.1. 기간 설정")
date = str(datetime.now().date()).replace('-', '')
print("기간: (%s)" %(date))

try:
  # 1.2. 티커별 종합정보조회
  print("1.2. 티커별 종합정보조회")
  sql = """
  SELECT BOLMFI.*, CAP.cap
  FROM (
    SELECT BOL.*, MFI.tp, MFI.mfi, MFI.mfi_diff
    FROM (
      SELECT * 
      FROM boll 
      WHERE date = %s
      AND period = 20
    ) AS BOL
    JOIN (
      SELECT ticker, tp, mfi, mfi_diff
      FROM mfi
      WHERE date = %s
      AND period = 10
    ) AS MFI
    ON (BOL.ticker = MFI.ticker)
  ) AS BOLMFI
  JOIN (
    SELECT ticker, cap
    FROM cap
  ) AS CAP
  ON (BOLMFI.ticker = CAP.ticker)    
  ORDER BY CAP.cap DESC
  """ % (date, date)
  cursor.execute(sql)
  sumDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'close', 4:'low', 5:'medium', 6:'high', 7:'bandWidth', 8:'position', 9: 'tp', 10: 'mfi', 11: 'mfi_diff', 12: 'name', 13: 'market', 14:'cap'})
  sumDf.set_index('ticker', drop=True, inplace=True)

  for idx in sumDf.index:
    row = sumDf.loc[idx].copy()
    ticker = row.name
    ############################
    ## BID STEP    
    if row['position'] <= position_bid and row['mfi'] <= mfi_bid and row['mfi'] > 0 and row['mfi_diff'] > 0:
      # 1.3. 매수기회포착
      print("1.3. 매수기회포착 (%s)" %(ticker))
      # 1.4. 매수여부확인
      print("1.4. 매수여부확인 (%s)" %(ticker))      
      if ticker in basketDf.index:
        print("[INFO] 이미 매수한 종목입니다. (%s)" %(ticker))
        continue
      # 매수
      elif ticker not in basketDf.index and len(basketDf) < step:
        # 1.5. 매수
        print("1.5. 매수 (%s)" %(ticker))      
        print("[INFO] 매수를 진행합니다. (%s)" %(ticker))
        # 1.5.1. 키움증권매수

        # 1.5.2. 매수내역기록
        print("1.5.2. 거래내역기록 (%s, %s)" %(date, ticker))    
        db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
        cursor = db.cursor()
        cursor.execute("INSERT INTO bid_basket (date, ticker, price, volume, mfi, position) VALUES (%s, %s, %s, %s, %s, %s)" %(date, ticker, row['close'], "", row['mfi'], row['position']))
        cursor.execute("INSERT INTO history (date, ticker, status, bid_price, ask_price, volume, pnl, realized_pnl) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)" %(date, ticker, 'bid', row['close'], "", "volume", "", ""))
        db.commit()
    if ticker in basketDf.index:
      info = basketDf.loc[ticker]
      bidPrice = info['bid_price']
      volume = info['volume']
      pnl = (1 - (bidPrice / row['close'])) * 100
      ############################
      ## ASK STEP    
      if row['position'] >= position_ask or row['mfi'] >= mfi_ask:
        # 1.6. 매도기회포착
        print("1.6. 매도기회포착 (%s)" %(ticker))
        # 1.7. 매도
        print("1.7. 매도 (%s)" %(ticker))      
        print("[INFO] 매도를 진행합니다. (%s)" %(ticker))
        # 1.7.1. 키움증권매도
        
        # 1.7.2. 매도내역기록
        db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
        cursor = db.cursor()
        print("1.7.2. 거래내역기록 (%s, %s)" %(date, ticker))      
        cursor.execute("DELETE FROM bid_basket WHERE ticker = %s" %(ticker))
        cursor.execute("INSERT INTO history (date, ticker, status, bid_price, ask_price, volume, pnl, realized_pnl) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)" %(date, ticker, 'ask', bidPrice, row['close'], volume, pnl, volume * pnl * 0.01))
        db.commit()
except Exception as ex1:
  print('ex1', traceback.format_exc(), ex1)

db.close()